In [23]:
# Now -- decision tree, random forests, boostoing, etc SVM
# logistic regression
# x train, validate, test
# 
# do I need to regularlize?
#
# start with tech, then for all entries in my database 

In [24]:
# useful links
# https://github.com/jakevdp/sklearn_scipy2013/blob/master/rendered_notebooks/06.1_validation_and_testing.ipynb
# http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html#sklearn.metrics.precision_recall_fscore_support
# http://scikit-learn.org/stable/modules/ensemble.html
# http://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html

In [210]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
from IPython.display import display
from IPython.core.pylabtools import getfigs
from sklearn import linear_model 
import seaborn as sns

In [26]:
%matplotlib qt

In [27]:
saveas_path = '/Users/jamie/insight data/figures/'

In [33]:
outcomes_df = pd.read_pickle('outcomes_df.save')

In [50]:
subset_predictor_df = pd.read_pickle('subset_predictor.save')
subset_predictor_df.head()

,Goal,Campaign_days,Title_n_words,Blurb_n_words,Month,Monday_day,Week_day,Hour,N_rewards,Min_reward,Median_reward,Max_reward,Reward_ship_days,Ships_intn,N_links,N_figs,Body_n_words,N_creator_backed
0,5000,30,2,17,8,12,6,20,8,10,175,5000,110,0,1,0,812,18
1,10000,30,8,21,6,30,5,14,10,5,125,5000,31,0,5,1,792,7
2,18000,27,5,14,2,25,1,8,19,5,500,10000,4,1,0,0,540,3
3,7000,56,5,17,12,6,1,17,8,10,87,1000,56,0,0,0,372,4
4,8150,30,8,16,7,22,1,11,11,4,42,815,41,-1,1,1,1405,8


In [104]:
subset_predictor_no_backers_df = subset_predictor_df.ix[:,0:-1]

In [105]:
subset_predictor_no_backers_df.head()

,Goal,Campaign_days,Title_n_words,Blurb_n_words,Month,Monday_day,Week_day,Hour,N_rewards,Min_reward,Median_reward,Max_reward,Reward_ship_days,Ships_intn,N_links,N_figs,Body_n_words
0,5000,30,2,17,8,12,6,20,8,10,175,5000,110,0,1,0,812
1,10000,30,8,21,6,30,5,14,10,5,125,5000,31,0,5,1,792
2,18000,27,5,14,2,25,1,8,19,5,500,10000,4,1,0,0,540
3,7000,56,5,17,12,6,1,17,8,10,87,1000,56,0,0,0,372
4,8150,30,8,16,7,22,1,11,11,4,42,815,41,-1,1,1,1405


In [51]:
outcomes_df.head()

,Outcome
0,1
1,1
2,1
3,1
4,1


In [38]:
n_samples = np.shape(outcomes_df)[0]
print n_samples

133071


In [39]:
# now use training (60%), validatation (20%), and test (20%) sets 
i_order = np.random.permutation(n_samples)
train_max_i = int(round(n_samples*.6))
validate_max_i = int(round(n_samples*.8))

train_is = i_order[:train_max_i]
validate_is = i_order[train_max_i:validate_max_i]
test_is = i_order[validate_max_i:]

In [141]:
X = subset_predictor_df
X_subset = subset_predictor_no_backers_df
y = outcomes_df['Outcome'].values

In [144]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(X.ix[train_is,:],y[train_is])
y_1 = clf.predict(X.ix[test_is,:])

clf_no_backers = DecisionTreeClassifier(max_depth=5)
clf_no_backers.fit(X_subset.ix[train_is,:],y[train_is])
y_1_no_backers = clf_no_backers.predict(X_subset.ix[test_is,:])

y_test = y[test_is]

In [145]:
accuracy = float(sum(y_test == y_1))/len(test_is)
print accuracy

0.71289546855


In [146]:
accuracy = float(sum(y_test == y_1_no_backers))/len(test_is)
print accuracy

0.672540768017


In [147]:
clf.feature_importances_

array([ 0.29910194,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.11109607,  0.        ,
        0.03033502,  0.0068397 ,  0.        ,  0.        ,  0.00758141,
        0.00092517,  0.05170728,  0.49241341])

In [113]:
clf_no_backers.feature_importances_

array([ 0.38395754,  0.01449195,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.37326251,  0.        ,
        0.01975522,  0.00916103,  0.        ,  0.        ,  0.12001353,
        0.0028102 ,  0.07654803])

In [114]:
print X_subset.columns

Index([u'Goal', u'Campaign_days', u'Title_n_words', u'Blurb_n_words', u'Month',
       u'Monday_day', u'Week_day', u'Hour', u'N_rewards', u'Min_reward',
       u'Median_reward', u'Max_reward', u'Reward_ship_days', u'Ships_intn',
       u'N_links', u'N_figs', u'Body_n_words'],
      dtype='object')


In [97]:
# calc F score -- 
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html#sklearn.metrics.precision_recall_fscore_support

In [98]:
# n backed is very important. what if I take it out? 
# how do trees handle mulicolinearlity? 

In [149]:
# now just try a few depths
# later separate validation and test
from sklearn.tree import DecisionTreeClassifier
for depth in range(3,20):

    clf = DecisionTreeClassifier(max_depth=depth)
    clf.fit(X.ix[train_is,:],y[train_is])
    y_1 = clf.predict(X.ix[test_is,:])

    y_test = y[test_is]

    accuracy = float(sum(y_test == y_1))/len(test_is)
    print depth, round(accuracy,2)

3 0.7
4 0.71
5 0.71
6 0.72
7 0.72
8 0.72
9 0.72
10 0.72
11 0.71
12 0.71
13 0.7
14 0.69
15 0.69
16 0.68
17 0.67
18 0.67
19 0.67


In [150]:
sklearn.metrics.precision_recall_fscore_support(y_test, y_1)

(array([ 0.71812036,  0.59777302]),
 array([ 0.69553942,  0.62368186]),
 array([ 0.70664954,  0.61045266]),
 array([15424, 11190]))

In [121]:
11190.0/(15424+11190)

0.42045539941384236

In [152]:
# now try a random forests model
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=200, max_depth=8)
clf.fit(X.ix[train_is,:],y[train_is])
y_1 = clf.predict(X.ix[test_is,:])

y_test = y[test_is]

In [153]:
accuracy = float(sum(y_test == y_1))/len(test_is)
print accuracy

0.731644998873


In [154]:
print X.ix[train_is,:].values

[[7500 45 8 ..., 0 314 4]
 [30000 30 3 ..., 0 640 1]
 [35000 30 2 ..., 0 321 3]
 ..., 
 [3500 30 6 ..., 0 232 0]
 [10000 25 9 ..., 9 825 5]
 [2300 32 2 ..., 2 117 4]]


In [155]:
y[train_is]

array([1, 1, 0, ..., 0, 0, 0])

In [156]:
from sklearn import grid_search

rf = RandomForestClassifier()
parameters = {'n_estimators':[200, 300, 400],
              'max_depth':[5, 7, 9]}

# Warning: be sure your data is shuffled before using GridSearch!
clf_grid = grid_search.GridSearchCV(rf, parameters)
clf_grid.fit(X.ix[train_is,:],y[train_is])

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'n_estimators': [200, 300, 400], 'max_depth': [5, 7, 9]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)

In [157]:
rf_best = clf_grid.best_estimator_
print rf_best.n_estimators, rf_best.max_depth

400 9


In [229]:
import pickle
s = pickle.dumps(clf_grid.best_estimator_)

In [230]:
j = pickle.loads(s)

In [232]:
print 'accuracy = ', round(j.score(X.ix[test_is,:],y_test),4)

accuracy =  0.7353


In [162]:
y_rf_best = rf_best.predict(X.ix[test_is,:])

In [191]:
print 'accuracy = ', round(clf_grid.best_estimator_.score(X.ix[test_is,:],y_test),4)

 accuracy =  0.7353


In [164]:
sklearn.metrics.precision_recall_fscore_support(y_test,y_rf_best)

(array([ 0.75973462,  0.6975968 ]),
 array([ 0.79441131,  0.65370867]),
 array([ 0.77668611,  0.67494003]),
 array([15424, 11190]))

In [194]:
best_forest_feature_gini_importances = clf_grid.best_estimator_.feature_importances_

In [195]:
print best_forest_feature_gini_importances

[ 0.25309932  0.035963    0.00547985  0.00675903  0.00797654  0.00601723
  0.00326037  0.00743814  0.10437812  0.01721128  0.03259521  0.03694492
  0.02354042  0.00306146  0.05717988  0.0123472   0.06759398  0.31915407]


In [198]:
feature_importance_is = np.argsort(best_forest_feature_gini_importances)

In [205]:
n_features = len(feature_importance_is)
print n_features

18


In [199]:
print feature_importance_is

[13  6  2  5  3  7  4 15  9 12 10  1 11 14 16  8  0 17]


In [204]:
feature_importance_is_decending = feature_importance_is[-1::-1]
print feature_importance_is_decending

[17  0  8 16 14 11  1 10 12  9 15  4  7  3  5  2  6 13]


In [220]:
plt.close('all')
sns.set_style("whitegrid")

In [227]:
# next add ste
#std = np.std([tree.feature_importances_ for tree in forest.estimators_],
#             axis=0)

In [221]:
plt.rcParams.update({'font.size': 26})

fig = plt.figure()
plt.title("Feature importances")
plt.bar(range(n_features), best_forest_feature_gini_importances[feature_importance_is_decending],
       color="b", align="center")
plt.xticks(range(n_features), X.columns[feature_importance_is_decending],rotation='vertical')
plt.xlim([-1, n_features])
plt.xlabel('Feature')
plt.ylabel('Feature importance (Gini importance)')
plt.show()

In [226]:
saveas_path = '/Users/jamie/insight_data/figures/'
plt.savefig(saveas_path + 'random forests importance june18.png',bbox_inches='tight',dpi=200) 

In [208]:
X.columns[feature_importance_is_decending]

Index([u'N_creator_backed', u'Goal', u'N_rewards', u'Body_n_words', u'N_links',
       u'Max_reward', u'Campaign_days', u'Median_reward', u'Reward_ship_days',
       u'Min_reward', u'N_figs', u'Month', u'Hour', u'Blurb_n_words',
       u'Monday_day', u'Title_n_words', u'Week_day', u'Ships_intn'],
      dtype='object')

In [175]:
# gradient boosting.
# https://github.com/jakevdp/sklearn_scipy2013/blob/master/rendered_notebooks/07.2_in_depth_trees_and_forests.ipynb
# then cross-validating Gradient Boosting

In [167]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=200, max_depth=9)
clf.fit(X.ix[train_is,:],y[train_is])

y_fit_boosted = clf.predict(X.ix[test_is,:])

In [185]:
print 'accuracy = ', round(clf.score(X.ix[test_is,:],y_test),4)

accuracy =  0.7393


In [179]:
clf.feature_importances_

array([ 0.11716369,  0.04838925,  0.03879327,  0.05025958,  0.04017738,
        0.05211624,  0.02574621,  0.04906849,  0.05664468,  0.02902795,
        0.06411725,  0.05657965,  0.08528523,  0.01508124,  0.03612113,
        0.04107662,  0.12054399,  0.07380814])

In [180]:
X.columns

Index([u'Goal', u'Campaign_days', u'Title_n_words', u'Blurb_n_words', u'Month',
       u'Monday_day', u'Week_day', u'Hour', u'N_rewards', u'Min_reward',
       u'Median_reward', u'Max_reward', u'Reward_ship_days', u'Ships_intn',
       u'N_links', u'N_figs', u'Body_n_words', u'N_creator_backed'],
      dtype='object')